[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sajitheranda/sign_language_web/blob/main/sign_backend/Signify_Backend.ipynb)

# Model

## Setup

In [1]:
!git clone --branch classifier https://github.com/chamodAchintha/Signify.git

Cloning into 'Signify'...
remote: Enumerating objects: 1439, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 1439 (delta 6), reused 10 (delta 6), pack-reused 1422 (from 1)
Receiving objects: 100% (1439/1439), 74.17 MiB | 30.06 MiB/s, done.
Resolving deltas: 100% (1086/1086), done.


In [2]:
%cd /content/Signify/Stochastic-Transformer-Networks

/content/Signify/Stochastic-Transformer-Networks


In [3]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/

## Load Models & Configs

In [1]:
%cd /content/Signify/Stochastic-Transformer-Networks

/content/Signify/Stochastic-Transformer-Networks


In [2]:
# Translation
!gdown 1UbS59BVb7tRzrbKDT64k3otoMRcYiWek -O /content/Signify/Stochastic-Transformer-Networks/SavedModels/translation/
!gdown 1dJ0AE_W9reoXrCkzGvmq3spKgEYMrMx2 -O /content/Signify/Stochastic-Transformer-Networks/SavedModels/translation/

# Classification
!gdown 1UzRJLMTWK9e7fYDUfkynnyPmzCA5EdHP -O /content/Signify/Stochastic-Transformer-Networks/SavedModels/classification/
!gdown 1W-PP3CPf5VVhQ54a0zB5LFbMZ3SS3eln -O /content/Signify/Stochastic-Transformer-Networks/SavedModels/classification/

Downloading...
From (original): https://drive.google.com/uc?id=1UbS59BVb7tRzrbKDT64k3otoMRcYiWek
From (redirected): https://drive.google.com/uc?id=1UbS59BVb7tRzrbKDT64k3otoMRcYiWek&confirm=t&uuid=2bcb27b0-8465-420d-b8b6-a3230c7881e5
To: /content/Signify/Stochastic-Transformer-Networks/SavedModels/translation/best_model.pth
100% 3.32G/3.32G [00:33<00:00, 97.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dJ0AE_W9reoXrCkzGvmq3spKgEYMrMx2
To: /content/Signify/Stochastic-Transformer-Networks/SavedModels/translation/translation_config.yaml
100% 1.94k/1.94k [00:00<00:00, 8.21MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1UzRJLMTWK9e7fYDUfkynnyPmzCA5EdHP
From (redirected): https://drive.google.com/uc?id=1UzRJLMTWK9e7fYDUfkynnyPmzCA5EdHP&confirm=t&uuid=7d31ae38-b903-471b-8f1d-fe6544b752c1
To: /content/Signify/Stochastic-Transformer-Networks/SavedModels/classification/best_model.pth
100% 175M/175M [00:01<00:00, 96.1MB/s]
Downloading...
From: https://drive.goog

In [3]:
import os
import logging
from sys import platform

import torch
import torch.nn.functional as F
from transformers import MBart50TokenizerFast
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

from signjoey.helpers import load_config, make_logger
from signjoey.classification_model import ClassificationModel
from signjoey.sinhala_sentence.translation_model import SinhalaSignTranslationModel
from signjoey.sinhala_sentence.search import greedy_decode


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [4]:
logger = logging.getLogger(__name__)
if not logger.handlers:
    logger.setLevel(level=logging.DEBUG)
    fh = logging.FileHandler("/content/inference.log")
    fh.setLevel(level=logging.DEBUG)
    logger.addHandler(fh)
    formatter = logging.Formatter("%(asctime)s %(message)s")
    fh.setFormatter(formatter)
    if platform == "linux":
        sh = logging.StreamHandler()
        sh.setLevel(logging.INFO)
        sh.setFormatter(formatter)
        logging.getLogger("").addHandler(sh)
    logger.info("Hello! This is Joey-NMT.")

INFO:__main__:Hello! This is Joey-NMT.
2025-05-21 19:34:27,049 Hello! This is Joey-NMT.


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Device: {device}")

INFO:__main__:Device: cuda
2025-05-21 19:34:27,055 Device: cuda


## Translation

In [6]:
%cd /content/Signify/Stochastic-Transformer-Networks

/content/Signify/Stochastic-Transformer-Networks


In [7]:
translation_config_path = "/content/Signify/Stochastic-Transformer-Networks/SavedModels/translation/translation_config.yaml"
translation_config = load_config(translation_config_path)
t_train_config = translation_config['training']

tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", tgt_lang=translation_config['data'].get('tgt_language', "si_LK"))

# Load model
translation_model = SinhalaSignTranslationModel(translation_config, logger)

translation_checkpoint_path = os.path.join(translation_config["training"]["model_dir"], 'best_model.pth')
translation_model.load_state_dict(torch.load(translation_checkpoint_path, map_location=device)['model_state_dict'])
translation_model.to(device)
translation_model.eval()

logger.info(f"Translation Model loaded for Inference from {translation_checkpoint_path}.")

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
2025-05-21 19:34:33,067 Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

INFO:__main__:creating the Translation model...
2025-05-21 19:34:38,605 creating the Translation model...
INFO:__main__:Graph Convolution Embeddings: False
2025-05-21 19:34:38,607 Graph Convolution Embeddings: False


512
512
512


INFO:__main__:Load model weights from a checkpoint: False
2025-05-21 19:34:38,881 Load model weights from a checkpoint: False
INFO:__main__:Freezed all decoder layers
2025-05-21 19:34:38,885 Freezed all decoder layers
INFO:__main__:Translation Model loaded for Inference from ./SavedModels/translation/best_model.pth.
2025-05-21 19:35:05,825 Translation Model loaded for Inference from ./SavedModels/translation/best_model.pth.


In [8]:
def prepare_inference_translation_input(keypoints: torch.Tensor, encoder_seq_len: int = 40):
    frame_count = keypoints.size(0)

    if frame_count < encoder_seq_len:
        # Padding
        padding = torch.zeros(encoder_seq_len - frame_count, keypoints.size(1))
        padded_keypoints = torch.cat([keypoints, padding], dim=0)
        keypoints_mask = torch.cat([
            torch.ones(frame_count, dtype=torch.long),
            torch.zeros(encoder_seq_len - frame_count, dtype=torch.long)
        ])
    elif frame_count == encoder_seq_len:
        padded_keypoints = keypoints
        keypoints_mask = torch.ones(encoder_seq_len, dtype=torch.long)
    else:
        raise ValueError(f"frame_count ({frame_count}) > encoder_seq_len ({encoder_seq_len})")

    # Add batch dimension
    return {
        "keypoints": padded_keypoints.unsqueeze(0).float(),         # [1, seq_len, 204]
        "keypoints_mask": keypoints_mask.unsqueeze(0).long()        # [1, seq_len]
    }


In [9]:
def get_translation_prediction(keypoints):
    with torch.no_grad():
        keypoints = prepare_inference_translation_input(sample_input)['keypoints'].to(device)
        keypoints_mask = prepare_inference_translation_input(sample_input)['keypoints_mask'].to(device)

        # Perform greedy decoding
        encoder_output = translation_model.encode(keypoints, keypoints_mask)[0]
        decoded_sequences = greedy_decode(
            src_mask=keypoints_mask,
            bos_index=tokenizer.lang_code_to_id.get(translation_config['data']['tgt_language']),
            eos_index=tokenizer.eos_token_id,
            max_output_length=translation_config['data']['max_sent_length'],
            decoder=translation_model.decoder,
            encoder_output=encoder_output,
            device=device
        )

        return {'text' : tokenizer.batch_decode(decoded_sequences, skip_special_tokens=True)[0].strip()}


In [10]:
# Sample Test
sample_input = torch.rand([31, 204])
get_translation_prediction(sample_input)

{'text': 'පුතා බ බොනවා.'}

## Classification

In [11]:
%cd /content/Signify/Stochastic-Transformer-Networks

/content/Signify/Stochastic-Transformer-Networks


In [12]:
classification_config_path = "/content/Signify/Stochastic-Transformer-Networks/SavedModels/classification/classification_config.yaml"
classification_config = load_config(classification_config_path)

# Load the checkpoint
classification_checkpoint_path = os.path.join(classification_config["training"]["model_dir"], 'best_model.pth')
classification_checkpoint = torch.load(classification_checkpoint_path, weights_only=False, map_location=device)

label_encoder = classification_checkpoint['label_encoder']

# Load model
classification_model = ClassificationModel(classification_config, logger)
classification_model.load_state_dict(classification_checkpoint['model_state_dict'])

classification_model.to(device)
classification_model.eval()

logger.info(f"Classification model loaded for inference from {classification_checkpoint_path}.")

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
INFO:__main__:creating the classification model...
2025-05-21 19:35:27,263 creating the classification model...
INFO:__main__:Graph Convolution Embeddings: False
2025-05-21 19:35:27,264 Graph Convolution Embeddings: False


512
512
512


INFO:__main__:Classification model loaded for inference from ./SavedModels/classification/best_model.pth.
2025-05-21 19:35:27,471 Classification model loaded for inference from ./SavedModels/classification/best_model.pth.


In [13]:
def prepare_classification_input(keypoints: torch.Tensor):
    seq_length = classification_config['data']['seq_length']
    frame_count = keypoints.size(0)

    if frame_count < seq_length:
        padding = torch.zeros(seq_length - frame_count, keypoints.size(1))
        padded_keypoints = torch.cat([keypoints, padding], dim=0)
        mask = torch.cat([
            torch.ones(frame_count, dtype=torch.bool),
            torch.zeros(seq_length - frame_count, dtype=torch.bool)
        ])
    elif frame_count == seq_length:
        padded_keypoints = keypoints
        mask = torch.ones(seq_length, dtype=torch.bool)
    else:
        raise ValueError(f"frame_count ({frame_count}) > seq_length ({seq_length})")

    return {
        "keypoints": padded_keypoints.unsqueeze(0).float(),  # [1, seq_length, 204]
        "mask": mask.unsqueeze(0)                             # [1, seq_length]
    }


In [14]:
def get_classification_prediction(keypoints):

    with torch.no_grad():
        inputs = prepare_classification_input(keypoints)
        data = inputs['keypoints'].to(device)
        mask = inputs['mask'].unsqueeze(1).expand(-1, 1, -1).to(device)

        output = classification_model(data, mask)

        probs = F.softmax(output, dim=1)
        top_probs, top_indices = torch.topk(probs, k=3, dim=1)

        top_classes = label_encoder.inverse_transform(top_indices[0].cpu().numpy())

        # Numbered predictions
        return {
            i + 1: (gloss.item(), round(prob.item(), 4))
            for i, (gloss, prob) in enumerate(zip(top_classes, top_probs[0]))
        }


In [15]:
# Test classification
sample_input = torch.rand([31, 204])
get_classification_prediction(sample_input)

{1: ('රියදුරු', 0.1991), 2: ('බිස්කට්', 0.1309), 3: ('බෑණා', 0.0749)}

# Backend

In [16]:
!pip install fastapi uvicorn pyngrok nest-asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00


In [17]:
from fastapi import FastAPI
from pydantic import BaseModel
import torch
import nest_asyncio
from pyngrok import ngrok
import uvicorn

In [18]:
!ngrok authtoken 2xOLBPcSEHDtC7wUqTmy3xGiqwK_2zprxCpK5w1aj43KwpVvQ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
class KeypointInput(BaseModel):
    keypoints: list[list[float]]  # Tensor as nested list: [frames, 204]

# ----- Start FastAPI app -----
app = FastAPI()

@app.post("/sign_classify")
def classify_sign(input_data: KeypointInput):
    keypoints_tensor = torch.tensor(input_data.keypoints, dtype=torch.float32)
    return get_classification_prediction(keypoints_tensor)

@app.post("/sign_translate")
def translate_sign(input_data: KeypointInput):
    keypoints_tensor = torch.tensor(input_data.keypoints, dtype=torch.float32)
    return get_translation_prediction(keypoints_tensor)

# ----- Start ngrok -----
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)

# Allow running uvicorn inside Colab
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)


Public URL: https://0c25-34-74-154-228.ngrok-free.app


INFO:     Started server process [3617]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     192.248.2.10:0 - "POST /sign_translate HTTP/1.1" 200 OK
INFO:     192.248.2.10:0 - "POST /sign_classify HTTP/1.1" 200 OK
INFO:     192.248.2.10:0 - "POST /sign_translate HTTP/1.1" 200 OK
INFO:     192.248.2.10:0 - "POST /sign_translate HTTP/1.1" 200 OK
